In [3]:
%load_ext cython

### def vs cdef vs cpdef

In [4]:
%%cython 

def inc(int num, int offset):
    return num + offset

def inc_seq(seq, offset):
    result = []
    for val in seq:
        res = inc(val, offset)
        result.append(res)
    return result

In [12]:
%%timeit
a = range(4)
inc_seq(a, 3)

291 ns ± 0.907 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [13]:
%%cython 

cdef int fast_inc(int num, int offset):
    return num + offset

def fast_inc_seq(seq, offset):
    result = []
    for val in seq:
        res = fast_inc(val, offset)
        result.append(res)
    return result

In [14]:
%%timeit
a = range(4)
fast_inc_seq(a, 3)

189 ns ± 1.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [15]:
%%cython

cpdef fast_inc(int num, int offset):
    return num + offset

def inc_seq(seq, offset):
    result = []
    for val in seq:
        res = fast_inc(val, offset)
        result.append(res)
    return result

In [17]:
%%timeit
a = range(4)
inc_seq(a, 3)

196 ns ± 0.654 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [19]:
fast_inc(1, 3)  # With cpdef, we can access `fast_inc` from Python.

4

# Hamming Distance

In [21]:
def hamming_sum(s0, s1):
    if len(s0) != len(s1):
        raise ValueError()
    return sum(c0 != c1 for (c0, c1) in zip(s0, s1))

def hamming_loop(s0, s1):
    if len(s0) != len(s1):
        raise ValueError()
    count = 0
    for i in range(len(s0)):
        count += (s0[i] != s1[i])
    return count

Let's try to make this code faster using Cython.

In [29]:
%%cython
def hamming_sum_(char *s0, char *s1):
    if len(s0) != len(s1):
        raise ValueError()
    return sum(c0 != c1 for (c0, c1) in zip(s0, s1))

In [31]:
%%timeit
hamming_sum_(b"qwertyuio", b"asdfghjkl")

470 ns ± 3.96 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [32]:
%%cython
def hamming_sum(char *s0, char *s1):
    if len(s0) != len(s1):
        raise ValueError()
        
    cdef int count = 0
    cdef int i
    cdef int N = len(s0)
    
    for i in range(N):
        count += (s0[i] != s1[i])
        
    return count

In [33]:
%%timeit
hamming_sum(b"qwertyuio", b"asdfghjkl")

46.2 ns ± 0.285 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


# Levenshtein Distance

In [34]:
def levenshtein_distance(s, t):
    return lev(s, len(s), t, len(t))

def lev(s, len_s, t, len_t):
    if len_s == 0 or len_t == 0:
        return len_s or len_t
    return min(lev(s, len_s-1, t, len_t  ) + 1,
               lev(s, len_s  , t, len_t-1) + 1,
               lev(s, len_s-1, t, len_t-1) + cost(s, len_s, t, len_t)) 

def cost(s, len_s, t, len_t):
    return s[len_s-1] != t[len_t-1]

In [36]:
%%timeit
levenshtein_distance("qwertyuio", "asdfghjkl")

326 ms ± 3.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%%cython
def levenshtein_distance_cy(char *s, char *t):
    return lev(s, len(s), t, len(t))

cdef lev(char *s, int len_s, char *t, int len_t):
    if len_s == 0 or len_t == 0:
        return len_s or len_t
    return min(lev(s, len_s-1, t, len_t  ) + 1,
               lev(s, len_s  , t, len_t-1) + 1,
               lev(s, len_s-1, t, len_t-1) + cost(s, len_s, t, len_t)) 

cdef cost(char *s, int len_s, char *t, int len_t):
    return s[len_s-1] != t[len_t-1]

In [43]:
%%timeit
levenshtein_distance_cy(b"qwertyuio", b"asdfghjkl")

24.5 ms ± 84.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
